<a href="https://colab.research.google.com/github/junsung2001/machine-learing/blob/main/250416_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 앙상블

## 랜덤포레스트

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('https://bit.ly/wine-date')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_jobs=-1, random_state=42) #랜덤 포레스트 변수를 rf로 지정
scores = cross_validate(rf, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [ ]:
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [ ]:
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42) # 부트스트랩 샘플에 포함되지 않는 남는 샘플로 평가. Out of Bag = oob_score

rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


## 엑스트라트리

In [ ]:
from sklearn.ensemble import ExtraTreesClassifier

et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target, return_train_score=True, n_jobs=-1) #return_train_score 학습 데이터셋을 가지고 평가

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [ ]:
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


## 그라디언트 부스팅

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [ ]:
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [ ]:
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15887763 0.6799705  0.16115187]


## 히스토그램 기반 부스팅

In [ ]:
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier

hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

/usr/local/lib/python3.11/dist-packages/sklearn/experimental/enable_hist_gradient_boosting.py:19: UserWarning: Since version 1.0, it is not needed to import enable_hist_gradient_boosting anymore. HistGradientBoostingClassifier and HistGradientBoostingRegressor are now stable and can be normally imported from sklearn.ensemble.
  warnings.warn(


0.9321723946453317 0.8801241948619236


In [ ]:
hgb.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [ ]:
hgb.score(test_input, test_target)

0.8723076923076923

#### XGBoost

In [ ]:
from xgboost import XGBClassifier

xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9558403027491312 0.8782000074035686


#### LightGBM

In [ ]:
from lightgbm import LGBMClassifier

lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target, return_train_score=True, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.935828414851749 0.8801251203079884


# 문제 : 새로운 데이터셋으로 RandomForest, XGBoost, LightGBM 성능 비교

In [ ]:
from sklearn.datasets import load_breast_cancer

cancer = load_breast_cancer() # array
c_data = cancer.data
c_target = cancer.target

# 이후 코드는 Wine 데이터셋과 동일하게 진행

In [24]:
c_data.shape

(569, 30)

In [38]:
c_target.shape

(569,)

In [52]:
# 데이터셋 분류
train_input, test_input, train_target, test_target = train_test_split(c_data, c_target, test_size=0.2, random_state=42)

In [53]:
# 랜덤포레스트 모델 정의
rf_model = RandomForestClassifier(n_jobs=-1, random_state=42)

In [54]:
# 랜덤포레스트 교차 검증 및 교차 검증 각 폴드의 학습 데이터 점수 평균, 각 폴드의 테스트 데이터 점수 평균
scores = cross_validate(rf_model, c_data, c_target, return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

1.0 0.9560937742586555


In [44]:
# 실제 모델 학습
rf_model.fit(c_data,c_target)

# 실제 모델 평가
rf_model= RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf_model.fit(c_data, c_target)
print(rf_model.oob_score_)

0.961335676625659


In [47]:
# XGBoost 모델 정의
xgb_model = XGBClassifier(tree_method='hist', random_state=42)
# 교차 검증
scores = cross_validate(xgb, c_data, c_target, return_train_score=True, n_jobs=-1)
# 교차 검증 평가 점수
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

1.0 0.9701288619779538


In [ ]:
# LightBGM 모델
lgb_model = LGBMClassifier(random_state=42)
# 교차 검증
scores = cross_validate(lgb, c_data, c_target, return_train_score=True, n_jobs=-1)
# 교차 검증 평가 점수
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

1.0 0.964835164835165
